In [3]:
import ee
from IPython.display import Image

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Get the image
image = ee.Image("NASA/NASADEM_HGT/001")

# Generate a URL for the image thumbnail
thumbnail_url = image.getThumbUrl({
    'dimensions': 400,  # Adjust the dimensions as needed
    'format': 'png'  # You can also use 'jpg' or other formats
})

# Display the thumbnail using IPython.display
Image(url=thumbnail_url)



Successfully saved authorization token.
